In [ ]:
import polars as pl
import pandas as pd
import preprocessing.transform_data as td
import preprocessing.feature_engineering as fe

In [ ]:
df_raw = pl.read_parquet("./data/2022_data_ST4000DM000.parquet")

In [ ]:
fe.search_faulty_drives(df_raw)

In [ ]:
df_raw = td.filter_out_inconsistent_drives(df_raw)

In [ ]:
Train, Test = td.custom_train_test_split(df_raw)

In [ ]:
len(Train)

In [ ]:
serial_nr_train = td.get_serial(df_raw)

In [ ]:
train_chunks = td.divide_chunks(Train, 1500)
test_chunks = td.divide_chunks(Test, 1500)

In [8]:
td.create_chunks(df_raw,Train, name="Train")

In [ ]:
td.create_chunks(df_raw,Test, name="Test")

In [ ]:
X_test = pd.DataFrame()
for i in range(1,5):
    df_new = pd.read_parquet(f"data/datachunks/Test_chunk{i}.parquet")
    X_test = pd.concat([X_test, df_new])


In [ ]:
X_test


In [ ]:
X_train = pd.DataFrame()
for i in range(1,11):
    df_new = pd.read_parquet(f"data/datachunks/Train_chunk{i}.parquet")
    X_train = pd.concat([X_train, df_new])


In [ ]:
X_train

In [ ]:
# code courtesy of jezrael on stackoverflow (https://stackoverflow.com/questions/46445965/reset-a-recurring-multiindex-in-pandas)
a1 = X_train.index.get_level_values(0).to_series()
a2 = a1.ne(a1.shift()).cumsum() - 1

mux = pd.MultiIndex.from_arrays([a2, X_train.index.get_level_values(1)], names=X_train.index.names)

X_train.index = mux

In [ ]:
# code courtesy of jezrael on stackoverflow (https://stackoverflow.com/questions/46445965/reset-a-recurring-multiindex-in-pandas)
a1 = X_test.index.get_level_values(0).to_series()
a2 = a1.ne(a1.shift()).cumsum() - 1

mux = pd.MultiIndex.from_arrays([a2, X_test.index.get_level_values(1)], names=X_test.index.names)

X_test.index = mux

In [ ]:
y_train = X_train.iloc[::14, :]["failing_in14days"].astype(int).reset_index(drop=True)

In [ ]:
y_test = X_test.iloc[::14, :]["failing_in14days"].astype(int).reset_index(drop=True)

In [ ]:
from sktime.transformations.panel.rocket import Rocket, MiniRocketMultivariate
from sktime.datatypes import convert,convert_to,mtype,check_raise
from sklearn.linear_model import SGDClassifier

In [ ]:
rocket = Rocket()

In [ ]:
col_list = td.feature_list(target=False)
X_train = X_train[col_list]

In [ ]:
minirocket_multi = MiniRocketMultivariate()
minirocket_multi.fit(X_train)


In [ ]:
X_train_transform = minirocket_multi.transform(X_train)

In [ ]:
#X_train2 = convert(df_rocket,from_type="pd-multiindex", to_type="df-list")

In [ ]:
#rocket.fit(X_train)

In [ ]:
#transformed_X = rocket.transform(X_train)